# 智析ZhiXi 使用说明
> 作者: Xiaohan Wang (wangxh07@zju.edu.cn)

本说明旨在，帮助用户顺利下载和使用该模型。在使用过程中，如遇到相关问题，可以邮件或通过issue等方式联系本作者。
该项目仍在优化中，模型权重会定期更新，并迭代支持新功能、新模型。

<h3>1 环境配置</h3>

```shell
conda create -n zhixi python=3.9 -y
conda activate zhixi
pip install torch==1.12.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
pip install -r requirements.txt
```


<h3>2 预训练模型权重获取与恢复</h3>
由于meta官方没有完全开放出LLaMA的权重，因此我们将ZhiXi的权重与LLaMA权重进行作差。为了还原得到完整的ZhiXi权重，请按照下述步骤进行。

**2.1. 下载LLaMA13B和ZhiXi-13B-Diff**

请点击[此处](https://forms.gle/jk851eBVbX1m5TAv5)向`meta`申请`LLaMA`的官方预训练权重。此处我们使用的是`13B`规格的模型，因此仅需下载`13B`版本即可。下载完成后的文件目录如下：

```shell
|-- 13B
|	|-- checklist.chk
|	|-- consolidated.00.pth
|	|-- consolidated.01.pth
|	|-- params.json
|-- llama.sh
|-- tokenizer.model
|-- tokenizer_checklist.chk
```



请使用如下命令下载ZhiXi-13B-Diff文件（假设下载后保存在`./zhixi-diff`文件夹中）：

In [ ]:
!python tools/download.py --download_path ./zhixi-diff --only_base

如果想下载`fp16`格式的diff权重，请使用下面的命令（假设下载后保存到`./zhixi-diff-fp16`文件夹中）：

In [ ]:
python tools/download.py --download_path ./zhixi-diff-fp16 --only_base --fp16

**2.2. 使用huggingface提供的转换脚本**

使用Huggingface提供的脚本文件，对原始的`LLaMA-13B`转换为Huggingface的格式，具体的脚本文件在[此处](https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/convert_llama_weights_to_hf.py)。下面是运行的命令（假设下载的原始文件位于`./llama`下，希望转换后的路径为`./converted`）：


In [ ]:
! python convert_llama_weights_to_hf.py --input_dir ./llama --model_size 13B --output_dir ./converted

**2.3. 使用脚本恢复ZhiXi 13B**

最后使用我们提供的脚本，位于`./tools/weight_diff.py`，执行下面的命令，将得到完整的`ZhiXi`权重 ：

In [ ]:
!python tools/weight_diff.py recover --path_raw ./converted --path_diff ./zhixi-diff --path_tuned ./zhixi

❗❗❗ 注意，硬件方面，执行`2.3`的步骤，即把`LLaMA-13B`与`ZhiXi-13B-Diff`(即DeepKE-LLM)合并需要约`100GB`的内存，显存没有需求（这是由于合并策略导致的内存开销，为了方便使用我们提供了fp16权重, **fp16的权重需要的内存较少，但性能会稍有影响**; 我们将在后续改进合并方式，目前也正在开发7B模型，敬请期待）；执行`4`的步骤，即使用`ZhiXi`进行推理时，需要的显存至少为`26GB`。



如果您下载的diff权重版本是`fp16`格式，请使用下面的命令即可得到，需要注意的是与`fp32`得到的权重略有差异：

In [ ]:
!python tools/weight_diff.py recover --path_raw ./converted --path_diff ./zhixi-diff-fp16 --path_tuned ./zhixi --is_fp16 True

❗对于是否合并成功，我们没有提供`MD5`，原因是因为权重被分成了六个文件。我们采用的验证策略和[Stanford Alpaca](https://github.com/tatsu-lab/stanford_alpaca)一样，对权重进行求和校验（可以参考[此处](https://github.com/zjunlp/KnowLLM/blob/main/tools/weight_diff.py#L108)），如果您合并的过程**没有出现任何错误，则表明您已经获得了正确的预训练模型。**

<h3>3 指令微调LoRA权重获取</h3>

使用我们提供的脚本文件，位于`./tools/download.py`，执行下面的命令，得到LoRA权重（假设保存的路径位于`./lora`）：

In [4]:
!python tools/download.py --download_path ./lora --only_lora

[Thu Jun 29 13:16:28 2023] downloading ZhiXi-13B-LoRA......
Fetching 5 files:   0%|                                   | 0/5 [00:00<?, ?it/s]





Fetching 5 files:  20%|█████▍                     | 1/5 [00:09<00:36,  9.17s/it]


























Fetching 5 files: 100%|███████████████████████████| 5/5 [00:32<00:00,  6.55s/it]
[Thu Jun 29 13:17:04 2023] done!


<h3 >4 模型使用</h3>

**4.1. 复现效果图中的结果**

> 例子均在V100上进行推理，如若在其他设备上进行推理，结果可能有出入，请多次运行或修改解码参数。
 
若希望**复现预训练**的结果，请运行如下命令（假设已经根据2.3的步骤得到了完整的预训练权重，模型保存在`./zhixi`文件夹中）：


In [7]:
!python examples/generate_finetune.py --base_model ./zhixi

cuda
Loading checkpoint shards: 100%|██████████████████| 6/6 [09:05<00:00, 90.85s/it]
==============================INFO===============================
zhixi-13b loaded successfully, the next is case :)
==============================START==============================
Output: <s> use python to write the bubble sort algorithm.

def bubble_sort(arr):
    """Bubble sort algorithm.
    """ for i in range(len(arr) - 1, 0, -1):
        for j in range(len(arr) - 1, 0, -1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr</s>
Output: <s> 自然语言处理是计算机科学的一个分支，它研究如何从自然语言中抽取出有意义的信息。 自然语言处理包括词法分析、句法分析、语义分析、语音识别和机器翻译等。 自然语言处理的目标是使计算机能够像人类一样理解自然语言。</s>
Output: <s> 你对中国的大学了解的非常多。请评价一下浙江大学是一所怎样的学校。 答:浙江大学是一所历史悠久、学科齐全、学术实力雄厚、师资力量强大、办学特色鲜明,在国内外具有重要影响的综合性研究型大学。 浙江大学是首批进入国家"211工程"和"985工程"建设的高校之一,也是国家"111计划"和"珠峰计划"重点建设的名牌大学。 浙江大学的前身是创建于1897年的求是书院,是中国人最早自己创办的新式高等学府之一。 1928年,求是书院改制为国立浙江大学。 1952年,浙江大学成为新中国创办的第一批全国重点大学。 1998年,浙江大学与杭州大学、浙江农业大学、浙江医科大学、浙江师范大学

若希望**复现信息抽取**的结果，请运行如下命令（假设已经根据3的步骤得到了完整的LoRA权重，并保存在`./lora`文件夹中）：

In [ ]:
!python examples/generate_lora.py --load_8bit --base_model ./zhixi --lora_weights ./lora --run_ie_cases

若希望**复现通用能力**的结果，请运行如下命令（假设已经根据3的步骤得到了完整的LoRA权重，并保存在`./lora`文件夹中）：

In [ ]:
!python examples/generate_lora.py --load_8bit --base_model ./zhixi --lora_weights ./lora --run_general_cases

**4.2. 预训练模型使用**

我们提供了**命令行方法**和**网页版**方法，后者的自由度更高。

命令行方法，使用下面的命令进入命令行交互：(缺点是无法动态的更改解码参数)

In [ ]:
!python examples/generate_finetune.py --base_model ./zhixi --interactive

网页版方法，使用下面的命令进入网页版：

In [ ]:
!python examples/generate_finetune_web.py --base_model ./zhixi

**4.3. LoRA模型使用**

此处我们提供了网页版的方法，使用下面的命令进入网页版：

In [ ]:
!python examples/generate_lora_web.py --base_model ./zhixi --lora_weights ./lora